# Twitter Sentiment Analysis - POC
---

## 6. Evaluate Several Models

Before throwing a bunch of models at the data, I first want to design a method to understand how models are learning. Etc... 

TODO: EXPLANATIONS FOR LEARNING CURVES


TODO: merge here? not a POC anymore if I'm not sampling down?

In [ ]:
import re
import os
import time
import json

import numpy as np
import pandas as pd

import cleanup_module_POC as Cmod

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [1]:
# time notebook
start_notebook = time.time()

# load minimally prepared X, y train subsets
raw_path = os.path.join("..","data","1_raw","sentiment140")
X = pd.read_csv(os.path.join(raw_path, "X_train.csv"))
y = pd.read_csv(os.path.join(raw_path, "y_train.csv"))

In [2]:
print(f'Dataset size: {len(X):0.0f}')
print(f'Target distribution: {sum(y["target"])/len(y):0.3f}')

Dataset size: 1197471
Target distribution: 0.499


In [3]:
X.head() # remember, original training indices saved in raw dir

,ID,username,tweet
0,1996787711,lifeischill,I went to frys to see andy and saw my BUD inst...
1,1985020349,knikkim,back home after brunch with church friends and...
2,1750930121,ginny9577,"@specialk0478 yeah she's part lab, part spanie..."
3,2063956079,kirstyhooper,kirsty is doing some ipd work!
4,1695806605,TwentyFour,@ScaryMommy Sure! My entire blogroll is ter...


In [4]:
# transform into arrays
X_array = np.array(X.iloc[:, 2]).ravel()
y_array = y.iloc[:,0].ravel()

In [5]:
# processed dir
proc_dir = os.path.join("..","data","3_processed","sentiment140")

try:
    # load sparce matrices
    X_train_transformed = sp.load_npz(os.path.join(proc_dir, "X_train_transformed.npz"))
except:
    preprocess_pipeline = Pipeline([
        ("document_to_wordcount", Cmod.DocumentToWordCounterTransformer()),
        ("wordcount_to_vector", Cmod.WordCounterToVectorTransformer()),
    ])
    start_pipeline = time.time()
    X_train_transformed = preprocess_pipeline.fit_transform(X_array)
    mins, secs = divmod(time.time() - start_pipeline, 60)
    print(f'Preprocessing pipeline time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

Preprocessing pipeline time: 12 minute(s) and 40 second(s).


In [6]:
X_train_transformed

<1197471x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 10503967 stored elements in Compressed Sparse Row format>

First training the full data on previous models.

In [7]:
NB_clf = MultinomialNB()
score = cross_val_score(NB_clf, X_train_transformed, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.756, total=   0.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] .................................... , score=0.756, total=   0.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV] .................................... , score=0.758, total=   0.7s
[CV]  ................................................................
[CV] .................................... , score=0.754, total=   0.7s
[CV]  ................................................................
[CV] .................................... , score=0.756, total=   0.7s
[CV]  ................................................................
[CV] .................................... , score=0.756, total=   0.7s
[CV]  ................................................................
[CV] .................................... , score=0.753, total=   0.8s
[CV]  ................................................................
[CV] .................................... , score=0.755, total=   0.8s
[CV]  ................................................................
[CV] .................................... , score=0.754, total=   0.8s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    7.7s finished


In [9]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.770, total=  19.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.4s remaining:    0.0s


[CV] .................................... , score=0.770, total=  19.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   38.4s remaining:    0.0s


[CV] .................................... , score=0.769, total=  19.1s
[CV]  ................................................................
[CV] .................................... , score=0.769, total=  18.8s
[CV]  ................................................................
[CV] .................................... , score=0.769, total=  17.9s
[CV]  ................................................................
[CV] .................................... , score=0.769, total=  19.3s
[CV]  ................................................................
[CV] .................................... , score=0.767, total=  19.3s
[CV]  ................................................................
[CV] .................................... , score=0.770, total=  19.1s
[CV]  ................................................................
[CV] .................................... , score=0.769, total=  19.7s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.2min finished


#### Save X_train_transformed in the processed folder.

In [11]:
import scipy.sparse as sp

save_dir = os.path.join("..","data","3_processed","sentiment140")
sp.save_npz(os.path.join(save_dir, 'X_train_transformed.npz'), X_train_transformed)

#### Learning Curves, n=1000, feed m

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

def plot_learning_curves_feed_m(model, X, y, Xname):
    """Plot the learning curves for train and validation sets
       , by adding more rows (m) and keeping the number of 
       features (n) constant.
       Args:
           model: a classifier
           X: sparce matrix of features
           y: target vector
           Xname: vectorization method
       Evaluates accuracy (TODO: implement other metrics)
    """
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                          test_size=0.2, 
                                                          random_state=42)
    # train
    # change data granularity (denominator) for more data points
    data_granularity = int(X_train.shape[0]/10)
    xs = []
    train_acc = []
    valid_acc = []
    for m in range(10, X_train.shape[0], data_granularity):       
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_valid_predict = model.predict(X_valid)
        train_acc.append(accuracy_score(y_train[:m], y_train_predict))
        valid_acc.append(accuracy_score(y_valid, y_valid_predict))
        xs.append(str(m-1))
    # plot
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_axes([0.0, 
                       # dynamically calculate number of x ticks
                       round(X_train.shape[0]/data_granularity, 0), 
                       0.4, 0.8])
    plt.plot(xs, np.sqrt(train_acc), 'r--', linewidth=.8, 
             label="train accuracy")
    plt.plot(xs, np.sqrt(valid_acc), 'b--', linewidth=.8, 
             label="validation accuracy")
    plt.xticks(rotation=45)
    plt.title(''.join(["Learning Rates for Training and Validation - ", 
                       Xname]), 
              fontsize=16)
    plt.legend(loc="lower right", fontsize=14)
    plt.xlabel("Training Set Size", fontsize=14)
    plt.ylabel("Accuracy", fontsize=14)
    plt.show()

In [ ]:
plot_learning_curves_feed_m(NB_clf, X_train_transformed, y_array, 
                     'Naive Bayes- BoW n=1000')

In [ ]:
plot_learning_curves(log_clf, X_train_transformed, y_array, 
                     'Logistic Regression (liblinear) - BoW n=1000')

#### Learning Curves, m=500k, feed n

Since it would take too long to transform the entire 1.2M dataset using a lot of features, we pair it down to ~500k since the learning curves for these models do not improve after about 100k anyway (currently).

In [ ]:
# sample down 
from sklearn.model_selection import train_test_split

X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.58, random_state=42)

In [ ]:
len(X)

In [ ]:
# transform into arrays
X_array = np.array(X.iloc[:, 2]).ravel()
y_array = y.iloc[:,0].ravel()

In [ ]:
preprocess_pipeline = Pipeline([
    ("document_to_wordcount", Cmod.DocumentToWordCounterTransformer()),
    ("wordcount_to_vector", Cmod.WordCounterToVectorTransformer(vocabulary_size=50000)),
])

start_pipeline = time.time()
X_train_transformed_50kfeat = preprocess_pipeline.fit_transform(X_array)
mins, secs = divmod(time.time() - start_pipeline, 60)
print(f'Preprocessing pipeline time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

In [ ]:
X_train_transformed_50kfeat

In [ ]:
def plot_learning_curves_feed_n(model, X, y, Xname):
    """Plot the learning curves for train and validation sets,
       by adding more features (n) and keeping the number of 
       rows (m) constant.
       Args:
           model: a classifier
           X: sparce matrix of features
           y: target vector
           Xname: vectorization method
       Evaluates accuracy (TODO: implement other metrics)
    """
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                          test_size=0.2, 
                                                          random_state=42)
    xs = []
    train_acc = []
    valid_acc = []
    for n in range(500, 50001, 4500):
        model.fit(X_train[:502937,:n], y_train)
        y_train_predict = model.predict(X_train[:502937,:n])
        y_valid_predict = model.predict(X_valid[:502937,:n])
        train_acc.append(accuracy_score(y_train, y_train_predict))
        valid_acc.append(accuracy_score(y_valid, y_valid_predict))
        xs.append(str(n-1))

    # plot
    fig = plt.figure(figsize=(16, 6))
    ax = fig.add_axes([0.0, 
                       # dynamically calculate number of x ticks
                       len(range(500, 50001, 4500)), 
                       0.4, 0.8])
    plt.plot(xs, np.sqrt(train_acc), 'r--', linewidth=.8, 
             label="train accuracy")
    plt.plot(xs, np.sqrt(valid_acc), 'b--', linewidth=.8, 
             label="validation accuracy")
    plt.xticks(rotation=45)
    plt.title(''.join(["Learning Rates for Training and Validation - ", 
                       Xname]), 
              fontsize=16)
    plt.legend(loc="lower right", fontsize=14)
    plt.xlabel("Training Set Size", fontsize=14)
    plt.ylabel("Accuracy", fontsize=14)
    plt.show()

In [ ]:
plot_learning_curves_feed_n(NB_clf, 
                            X_train_transformed_50kfeat, 
                            y_array, 
                            'Naive Bayes - BoW m=500k')

In [ ]:
start_LR_plot = time.time()
plot_learning_curves_feed_n(log_clf, 
                            X_train_transformed_50kfeat, 
                            y_array, 
                            'Logistic Regression - BoW m=500k')
mins, secs = divmod(time.time() - start_LR_plot, 60)
print(f'LR plot time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

In [ ]:
score = cross_val_score(NB_clf, X_train_transformed_50kfeat, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

In [ ]:
score = cross_val_score(log_clf, X_train_transformed_50kfeat, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

In [ ]:
# time notebook
mins, secs = divmod(time.time() - start_notebook, 60)
print(f'Total running time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

---